Add all imports here :)

In [7]:
import cv2
import numpy as np
import pyautogui
from PIL import ImageGrab
import tkinter as tk
from tkinter import ttk
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import time

DEF THE CARD DETECTION GUI

In [8]:
# GUI Class
class CardDetectorGUI:
    def __init__(self, master):
        self.master = master
        master.title("Card Detector")

        self.start_button = ttk.Button(master, text="Start Detection", command=self.start_detection)
        self.start_button.pack()

        self.stop_button = ttk.Button(master, text="Stop Detection", command=self.stop_detection)
        self.stop_button.pack()
        self.stop_button.config(state='disabled')

        self.status_label = ttk.Label(master, text="Status: Idle")
        self.status_label.pack()

        self.is_running = False

    def start_detection(self):
        self.is_running = True
        self.start_button.config(state='disabled')
        self.stop_button.config(state='normal')
        self.status_label.config(text="Status: Running")
        self.master.after(100, self.run_detection)

    def stop_detection(self):
        self.is_running = False
        self.start_button.config(state='normal')
        self.stop_button.config(state='disabled')
        self.status_label.config(text="Status: Stopped")

    def run_detection(self):
        if self.is_running:
            # Get screen size
            screen_width, screen_height = pyautogui.size()
            
            # REGION OF INTEREST V IMPORTANT
            roi_width = int(screen_width // 3)
            roi_height = int(screen_height)
            roi_left = 0
            roi_top = 0
            
            # Capture screen
            screen = np.array(ImageGrab.grab(bbox=(roi_left, roi_top, roi_left + roi_width, roi_top + roi_height)))
            screen = cv2.cvtColor(screen, cv2.COLOR_RGB2BGR)

            # Process the image
            corners = detect_card_corners(screen)
            result = draw_top_left_corner_boxes(screen.copy(), corners)
            corner_regions = extract_corner_regions(screen, corners)

            # Here you would run your model predictions on corner_regions

            # Display the result
            cv2.imshow('Card Detection', result)
            cv2.waitKey(1)

            self.master.after(100, self.run_detection)

CARD DETECTION AND IMAGE FUNCTIONS

In [9]:
def detect_card_corners(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    card_corners = []
    for contour in contours:
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
        
        if len(approx) == 4:  # Assuming cards are rectangular
            # Get bounding rectangle
            x, y, w, h = cv2.boundingRect(approx)
            
            # Check if the contour is large enough to be a card
            if w > 100 and h > 150:  # Adjust these thresholds as needed
                # Add only the top-left corner
                card_corners.append((x, y))
    
    return card_corners

In [10]:
def draw_top_left_corner_boxes(image, corners):
    for corner in corners:
        x, y = corner
        corner_width = 80  # Adjust this to ensure suit and number are included
        corner_height = 180  # Adjust this to ensure suit and number are included
        cv2.rectangle(image, (x, y), (x + corner_width, y + corner_height), (0, 255, 0), 2)
    return image

In [11]:
def extract_corner_regions(image, corners):
    corner_regions = []
    corner_width = 80  # Should match the width in draw_top_left_corner_boxes
    corner_height = 180  # Should match the height in draw_top_left_corner_boxes
    
    for corner in corners:
        x, y = corner
        corner_region = image[y:y+corner_height, x:x+corner_width]
        corner_regions.append(corner_region)
    
    return corner_regions

In [12]:
root = tk.Tk()
gui = CardDetectorGUI(root)
root.mainloop()